In [2]:
import time
import random
import pickle
import numpy as np 
import pandas as pd
import wandb

from tqdm.notebook import tqdm

from Utils import metrics, templates, utils

In [ ]:
wandb.login(key="b9d02b504b9d57ae7801f351e79da0162387f010")

In [3]:
## Configurações base

config = {
    "Arch" : "llama",
    "runtime": "ROCm llama.cpp v1.18.0",
    "dataset": "ml_100k",
    "nsu" : 12,
    "nci" :19,
    "lenlimit" :8,
    "test_run" : 5, # faz apenas 100 recomendações
    "print_results": True,
    "obs": "Dataset = filme (ano)"
}

In [4]:
## define o prompt template

prompt_template = templates.PROMPT_TEMPLATE_1
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
config.update({"prompt_format": prompt_format})

In [5]:
## Configuração do modelo 

# llama-3.2-3b-instruct

config.update({
    "model_name" :"llama-3.2-3b-instruct",
    "Quantization" : "Q8_0",
    "Temperature": 0.1,
    "System_prompt" : "Answer with just what was asked.",
    "max_tokens" : 4096,
})

In [12]:
# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K.json")
print(f'Quantidade de Usuários: {len(dataset)}')

Quantidade de Usuários: 943


In [7]:
id_list = list(range(0, len(dataset)))
#assert(len(id_list) == 943) # aqui é verificado se a lista possue exatamente essa quantidade

# Building indexes and similarity matrices for users and movies.
movie_idx = utils.build_moviename_index_dict(dataset)
user_sim_matrix = utils.build_user_similarity_matrix(dataset, movie_idx)
pop_dict = utils.build_movie_popularity_dict(dataset)
item_sim_matrix = utils.build_item_similarity_matrix(dataset)


results = {'config': config}

nsu = config["nsu"]
nci = config["nci"]
lenlimit = config["lenlimit"]

In [8]:
results['start_time'] = time.time()

# inicia a run do wandb

#run = wandb.init(project="test",config=config)

all_results = []

if config["test_run"]:
    id_list = id_list[:config["test_run"]] # Define a quantiadade que será processado

for i in tqdm(id_list, desc="Processando", unit="it"): 

    results[i] = {}

    watched_mv = dataset[i][0].split(' | ')[::-1]
    results[i]['ground_truth'] = dataset[i][-1]

    # Generate candidate items based on user filtering.
    #candidate_items = utils.sort_user_filtering_items(dataset, watched_mv, user_sim_matrix[i], nsu, nci)
    candidate_items = utils.sort_collaborative_user_filtering(target_user_id=i,
                                                             dataset=dataset,
                                                             user_similarity_matrix=user_sim_matrix,
                                                             num_users=nsu,
                                                             num_items=nci,
                                                             include_similar_user_GT=False,
                                                             debug=False)
    random.shuffle(candidate_items)

    results[i]['candidate_set'] = candidate_items

    # verifica se o ground_truth está no candidate_set
    results[i]['gt_in_candidate_set'] = "yes" if any(results[i]['ground_truth'].lower() in candidate.lower() for candidate in results[i]['candidate_set']) else "no"

    # pipeline 

    # STEP 1
    input_1 = prompt_template['Preference'].format(', '.join(watched_mv[-lenlimit:]))
    results[i]['input_1'] = input_1
    response = utils.query_lm_studio(config["model_name"],config["Temperature"],prompt_template['System_prompt'],input_1,config["max_tokens"])
    predictions_1 = response
    results[i]['predictions_1'] = predictions_1

    # STEP 2
    input_2 = prompt_template['Featured_movies'].format(', '.join(watched_mv[-lenlimit:]), predictions_1)
    results[i]['input_2'] = input_2
    response = utils.query_lm_studio(config["model_name"],config["Temperature"],prompt_template['System_prompt'],input_2,config["max_tokens"])
    predictions_2 = response
    results[i]['predictions_2'] = predictions_2

    # STEP 3
    input_3 = prompt_template['Recommendation'].format(', '.join(candidate_items),', '.join(watched_mv[-lenlimit:]), predictions_1, predictions_2)
    results[i]['input_3'] = input_3
    response = utils.query_lm_studio(config["model_name"],config["Temperature"],prompt_template['System_prompt'],input_3,config["max_tokens"])
    predictions_3 = response
    results[i]['predictions_3'] = predictions_3


    # STEP 4 - Formate the response
    input_4 = prompt_format['Prompt'].format(predictions_3)
    response = utils.query_lm_studio(config["model_name"],config["Temperature"],prompt_format['System_prompt'],input_4,config["max_tokens"])
    recommendations = response
    results[i]['recommendations'] = recommendations


    # Calculate metrics
    run_metrics = metrics.calculate_metrics(results[i]['recommendations'], results[i]['ground_truth'])    
    results[i]['hit'] = run_metrics['hit']
    results[i]['precision'] = run_metrics['precision']
    results[i]['recall'] = run_metrics['recall']
    results[i]['ndcg'] = run_metrics['ndcg']

    #run.log()

    if config["print_results"]:
        print(f"\n** User: {i}")
        print(f"** Ground Truth: {results[i]['ground_truth']}")
        print(f"** Candidate_set: {results[i]['candidate_set']}")
        #print(f"** Input Final: {input_3}")
        #print(f"** Predicao Final: {predictions_3}")
        print(f"** Recomendações: {results[i]['recommendations']}")
        print(f"** Hit: {results[i]['hit']}")
        print(f"** Precision: {results[i]['precision']}")
        print(f"** Recall: {results[i]['recall']}")
        print(f"** NDCG: {results[i]['ndcg']}")

results['end_time'] = time.time()
results['runtime'] = results['end_time'] - results['start_time']

# calculate average metrics
results['metrics'] = metrics.calculate_average_metrics(results)

# save dictionary to pickle file
arq_name = utils.save_result_to_pickle(results, config)

#run_name = run.name
#run.finish()
#wandb.finish()


Processando:   0%|          | 0/5 [00:00<?, ?it/s]


** User: 0
** Ground Truth: Aristocats, The (1970)
** Candidate_set: ['Air Force One (1997)', 'Mrs. Doubtfire (1993)', 'Scream (1996)', "One Flew Over the Cuckoo's Nest (1975)", 'Stand by Me (1986)', 'Liar Liar (1997)', 'Interview with the Vampire (1994)', 'Heathers (1989)', 'Batman (1989)', 'Executive Decision (1996)', 'E.T. the Extra-Terrestrial (1982)', 'That Thing You Do! (1996)', 'Heat (1995)', "Schindler's List (1993)", 'True Lies (1994)', 'Mission: Impossible (1996)', 'Die Hard: With a Vengeance (1995)', 'Clueless (1995)', 'Speed (1994)']
** Recomendações: The Shawshank Redemption; Seven Samurai; The Usual Suspects; Apocalypse Now; The Silence of the Lambs; Platoon; The Terminator; Raiders of the Lost Ark; The Princess Bride; Goodfellas
** Hit: 0
** Precision: 0.0
** Recall: 0.0
** NDCG: 0.0

** User: 1
** Ground Truth: River Wild, The (1994)
** Candidate_set: ['Usual Suspects, The (1995)', 'Primal Fear (1996)', 'Game, The (1997)', 'Trainspotting (1996)', 'Twelve Monkeys (1995)

In [9]:
results

{'config': {'Arch': 'llama',
  'runtime': 'ROCm llama.cpp v1.18.0',
  'dataset': 'ml_100k',
  'nsu': 12,
  'nci': 19,
  'lenlimit': 8,
  'test_run': 5,
  'print_results': True,
  'obs': 'Dataset = filme (ano)',
  'prompt_template': {'System_prompt': "You are a movie expert provide the answer for the question based on the given context. If you don't know the answer to a question, please don't share false information.",
   'Preference': '\n    ### MY WATCHED MOVIES LIST: {}.\n\n    ### QUESTION: Based on my watched movies list. Tell me what features are most important to me when selecting movies (Summarize my preferences briefly)?\n\n    ### ANSWER:\n    ',
   'Featured_movies': '\n\n    ### MY WATCHED MOVIES LIST: {}.\n\n    ### MY MOVIE PREFERENCES: {}.\n\n    ### QUESTION: Create an enumerated list selecting the five most featured movies from the watched movies according to my movie preferences.\n\n    ### ANSWER:\n    ',
   'Recommendation': '\n\n    ### CANDIDATE MOVIE SET: {}.\n\n 

In [ ]:
#arq_name = 'ml_100k-zs-nir-su12-ci19-llama-3.2-3b-instruct.pkl'

with open(f'Results/{arq_name}', 'rb') as f:
    data = pickle.load(f)

#data

In [ ]:
results = []
for key, value in data.items():
    if isinstance(key, int) and isinstance(value, dict):  # Pegando apenas os experimentos
        results.append({
            'Candidates': value.get('candidate_set', ''),
            'Ground Truth': value.get('ground_truth', ''),
            'gt_in_candidate_set': value.get('gt_in_candidate_set', ''),
            #'Input 1': value.get('input_1', ''),
            #'Predictions 1': value.get('predictions_1', ''),
            #'Input 2': value.get('input_2', ''),
            #'Predictions 2': value.get('predictions_2', ''),
            'Input 3': value.get('input_3', ''),
            'Predictions 3': value.get('predictions_3', ''),
        })

df_results = pd.DataFrame(results)

df_results


In [ ]:
system_prompt = "You are a data extraction assistant. Your task is to extract only the movie titles from the provided text and return them in a structured format, separated by semicolons. Do not include any extra text, explanations, or numbers—only the movie titles."
prompt = """Rewrite the following list of recommendations, extracting only the movie titles and separating them with a semicolon.
Original text:{}
""".format(df_results['Predictions 3'][0])

response = utils.query_lm_studio(config["model_name"],config["Temperature"],system_prompt,prompt,config["max_tokens"])

In [ ]:
df_results['Predictions 3'][0]

In [ ]:
response